In [2]:
import requests
import pandas as pd

def fetch_earthquake_data():
    #  USGS Earthquake API endpoint
    url = (
        "https://earthquake.usgs.gov/fdsnws/event/1/query"
        "?format=geojson&starttime=2024-01-01&minmagnitude=4&limit=1000"
    )

    print(" Fetching earthquake data from USGS API...")
    response = requests.get(url, timeout=15)  # timeout = 15s

    #  Step 1: Check if request succeeded
    if response.status_code != 200:
        print(f" Failed to fetch data. HTTP {response.status_code}")
        print("Response:", response.text[:200])
        return

    #  Step 2: Check if the body is not empty
    if not response.text.strip():
        print(" Empty response from the server — no data available.")
        return

    try:
        data = response.json()  # Try to decode JSON
    except Exception as e:
        print(" Failed to decode JSON:", e)
        print("Response (first 200 chars):", response.text[:200])
        return

    #  Step 3: Extract features safely
    if "features" not in data:
        print(" No 'features' field found in response. Data format changed?")
        return

    earthquakes = []
    for feature in data["features"]:
        props = feature.get("properties", {})
        geom = feature.get("geometry", {})
        coords = geom.get("coordinates", [None, None, None])
        earthquakes.append([
            props.get("time"),
            props.get("mag"),
            coords[0],  # longitude
            coords[1],  # latitude
            coords[2],  # depth
        ])

    if not earthquakes:
        print(" No earthquake records found.")
        return

    #  Step 4: Convert to DataFrame
    df = pd.DataFrame(
        earthquakes,
        columns=["Time", "Magnitude", "Longitude", "Latitude", "Depth"]
    )

    #  Step 5: Save to CSV
    df.to_csv("earthquake_data.csv", index=False)
    print(f" Saved {len(df)} earthquake records to 'earthquake_data.csv'")

if __name__ == "__main__":
    fetch_earthquake_data()


 Fetching earthquake data from USGS API...
 Saved 1000 earthquake records to 'earthquake_data.csv'
